In [78]:
# importing libraries

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.sql import text

----

### Number of samples 

-----

In [79]:
# defining the sample size
n = 1000

----

### Generating Customer Data Dataset

----

In [80]:
# creating a function to generate the dataset which contain customer details

def customer_data():

    # creating a customer_id in a range of 1001 to 2000
    customer_id = list(range(1001, 1001 + n))

    # adding customer_name with a prefix of cus_ and a range of 1 to n
    customer_name = [f'cus_{i}' for i in range(1, n + 1)]

    # adding customer the data is taken from 1940 to 2000
    dob = np.random.choice(pd.date_range(start='1/1/1940', end='1/1/2000'), n)

    # Convert to date-only format (without timestamp)
    dob = pd.Series(dob).dt.strftime('%Y-%m-%d')  # Convert each element to string format


    # gender of the customer
    sex = np.random.choice(['Male', 'Female', 'Other'], n)

    # nationality of the customer
    nationality = np.random.choice(['English', 'French', 'German', 'Italian', 'Spanish', 'Other'], n)


    # returning the dataset
    return pd.DataFrame(
        {
        'customer_id': customer_id,
        'customer_name': customer_name, 
        'dob': dob, 
        'sex': sex, 
        'nationality': nationality
        }
        )

In [81]:
# calling the function
df_customer = customer_data()
# printing the head of the dataset
df_customer.head()

,customer_id,customer_name,dob,sex,nationality
0,1001,cus_1,1961-03-05,Male,German
1,1002,cus_2,1971-01-09,Other,Spanish
2,1003,cus_3,1955-03-12,Male,Italian
3,1004,cus_4,1994-03-05,Other,Italian
4,1005,cus_5,1978-09-17,Female,French


----

### Introducing missing values in  `sex` and `nationality` columns

----

In [82]:

# Introduce missing values in `sex` column

# selecting 10 random indices
missing_indices_sex = np.random.choice(df_customer.index, size=10, replace=False)  
# assigning NaN to the selected indices in the `sex` column
df_customer.loc[missing_indices_sex, 'sex'] = np.nan 

# selecting 10 another random indices
missing_indices_nationality = np.random.choice(df_customer.index, size=10, replace=False)  
# assigning NaN to the selected indices in the `nationality` column
df_customer.loc[missing_indices_nationality, 'nationality'] = np.nan



----

### Saving the dataset to csv file

----

In [53]:

# saving the dataset to csv file
df_customer.to_csv('customer_data.csv', index=False)

# printing 
print('Data has been saved to customer_data.csv')

Data has been saved to customer_data.csv


----

### Generating Booking Data Dataset

----

In [54]:
# creating a function to generate the dataset which contain booking details

def booking_data():

    # creating a column booking_id in a range of 23450 to 24450
    booking_id = list(range(23450, 23450 + n))

    # creating a column customer_id in a range of 1001 to 2000
    customer_id = list(range(1001, 1001 + n))

    room_number = list(range(101, 101 +  n))

    # room_type of the booking
    room_type = np.random.choice(['Single', 'Double', 'Suite'], n)

    # Room rate per night (USD)
    room_rate = np.where(room_type == 'Single', 50, 
        np.where(room_type == 'Double', 75, 
        np.where(room_type == 'Suite', 150, 0)))
    
    # number of rooms booked
    rooms_booked = np.random.choice(range(1, 4), n)

    # number of persons in the booking

    num_persons = np.where(
    # Single: 1 person per room
    room_type == 'Single', rooms_booked, 
    # Double: 1-2 persons per room 
    np.where(room_type == 'Double', np.random.randint(1, 3, n) * rooms_booked, 
    # Suite: 1-4 persons per room
    np.random.randint(1, 5, n) * rooms_booked) 
    )

    # check-in date
    check_in_date = np.random.choice(pd.date_range(start='1/1/2023', end='1/1/2024'), n).astype('datetime64[D]')
    # check in time
    check_in_time = np.random.randint(0, 24*60, n).astype('timedelta64[m]')
    # check-in datetime
    check_in = check_in_date + check_in_time

    # adding column location id
    location_id = np.random.choice(range(1, 6), n)

    # check-out date
    check_out_date = check_in + np.random.choice(range(1, 11), n).astype('timedelta64[D]')
    # check out time
    check_out_time = np.random.randint(0, 24*60, n).astype('timedelta64[m]')
    # check-out datetime
    check_out = check_out_date + check_out_time

    # Format check-in and check-out times to show only hours and minutes
    check_in = pd.Series(check_in).dt.strftime('%Y-%m-%d %H:%M')
    check_out = pd.Series(check_out).dt.strftime('%Y-%m-%d %H:%M')

    # number of days stayed
    num_days = (pd.to_datetime(check_out) - pd.to_datetime(check_in)).dt.days

    # total cost of the booking
    total_cost = room_rate * rooms_booked * num_days
    
    # payment status
    payment_status = np.random.choice(['Paid', 'Pending'], n)

    # returning the dataset
    return pd.DataFrame(
        {
        'booking_id': booking_id,
        'customer_id': customer_id,
        'location_id': location_id,
        'room_number': room_number,
        
        'room_type': room_type,
        'room_rate': room_rate,
        'rooms_booked': rooms_booked,
        'num_persons': num_persons,
        'check_in': check_in,
        'check_out': check_out,
        'num_days_stayed': num_days,
        'total_cost': total_cost,
        'payment_status': payment_status
    }
    )

----
### Calling the function
----

In [55]:
# calling the function
df_booking = booking_data()

# preview the dataset
df_booking.head()


,booking_id,customer_id,location_id,room_number,room_type,room_rate,rooms_booked,num_persons,check_in,check_out,num_days_stayed,total_cost,payment_status
0,23450,1001,3,101,Single,50,3,3,2023-01-22 00:45,2023-01-26 05:13,4,600,Paid
1,23451,1002,5,102,Single,50,3,3,2023-09-07 06:57,2023-09-17 02:17,9,1350,Paid
2,23452,1003,2,103,Double,75,3,6,2023-02-06 21:22,2023-02-10 23:15,4,900,Paid
3,23453,1004,2,104,Double,75,2,4,2023-01-16 00:14,2023-01-19 09:26,3,450,Pending
4,23454,1005,3,105,Single,50,1,1,2023-03-04 08:36,2023-03-13 18:52,9,450,Pending


----

### Delibrately introducing duplicate records in booking details table on basis of check-in time and room_number
----

In [57]:
# Select 10 random records and duplicate check-in time while maintaining room_id
duplicate_records = df_booking.sample(10, random_state=0).copy()

# Assign new unique customer IDs to the duplicated records
duplicate_records['customer_id'] = range(df_booking['customer_id'].max() + 1, df_booking['customer_id'].max() + 11)
    
# Assign new unique booking IDs to the duplicated records
duplicate_records['booking_id'] = range(df_booking['booking_id'].max() + 1, df_booking['booking_id'].max() + 11)
    
# Ensure the room_number and check-in time remain the same
duplicate_records['room_number'] = duplicate_records['room_number']
duplicate_records['check_in'] = duplicate_records['check_in']
duplicate_records['room_type'] = duplicate_records['room_type']

# Reorder columns to match the original dataset    
duplicate_records = duplicate_records[['booking_id', 'customer_id', 'location_id', 'room_number', 'room_type', 
                                       'room_rate', 'rooms_booked', 'num_persons', 'check_in', 'check_out', 'num_days_stayed', 
                                       'total_cost', 'payment_status']]
    
# Append duplicated records to the dataset
df_booking = pd.concat([df_booking, duplicate_records], ignore_index=True)

In [61]:
# Select and print all 20 duplicate records
df_booking[df_booking.duplicated(subset=['check_in', 'room_number'], keep=False)]


,booking_id,customer_id,location_id,room_number,room_type,room_rate,rooms_booked,num_persons,check_in,check_out,num_days_stayed,total_cost,payment_status
27,23477,1028,4,128,Suite,150,2,8,2023-04-18 16:58,2023-04-21 05:51,2,600,Pending
231,23681,1232,1,332,Single,50,3,3,2023-06-30 04:12,2023-07-01 09:28,1,150,Pending
298,23748,1299,5,399,Suite,150,3,12,2023-08-08 16:27,2023-08-18 23:47,10,4500,Pending
306,23756,1307,2,407,Suite,150,2,4,2023-08-12 09:09,2023-08-16 16:00,4,1200,Paid
553,24003,1554,4,654,Single,50,2,2,2023-06-10 05:13,2023-06-20 18:13,10,1000,Pending
672,24122,1673,4,773,Suite,150,3,12,2023-03-02 16:52,2023-03-04 19:35,2,900,Pending
706,24156,1707,2,807,Single,50,3,3,2023-02-01 22:33,2023-02-05 11:54,3,450,Pending
859,24309,1860,5,960,Double,75,1,2,2023-08-13 05:54,2023-08-20 05:32,6,450,Pending
971,24421,1972,5,1072,Suite,150,2,8,2023-10-14 14:03,2023-10-25 02:04,10,3000,Paid
993,24443,1994,3,1094,Single,50,3,3,2023-03-31 12:31,2023-04-05 20:09,5,750,Paid


In [63]:
# saving the dataset to csv file
df_booking.to_csv('booking_data.csv', index=False)

# printing
print('Data has been saved to booking_data.csv')

Data has been saved to booking_data.csv


----

### Generating Location Details Dataset

----

In [64]:
def location_details():

    # location_id columns which shows unique location id for each hotel
    location_id = range(1,6)

    # hotel name
    hotel_name = ['AK Hotel', 'AK Hotel', 'AK Hotel', 'AK Hotel', 'AK Hotel']

    # city
    city = ['New York', 'London', 'Paris', 'Tokyo', 'Sydney']

    # country
    country = ['USA', 'UK', 'France', 'Japan', 'Australia']

    # returning the dataset
    return pd.DataFrame(
        {
            'location_id': location_id,
            'hotel_name': hotel_name,
            'city': city,
            'country': country
        }
    )   


In [65]:
# calling the function
df_location = location_details()
# setting index is location_id

# df_location = df_location.set_index('location_id')
# preview the dataset
df_location.head()

,location_id,hotel_name,city,country
0,1,AK Hotel,New York,USA
1,2,AK Hotel,London,UK
2,3,AK Hotel,Paris,France
3,4,AK Hotel,Tokyo,Japan
4,5,AK Hotel,Sydney,Australia


In [66]:
# saving the dataset to csv file
df_location.to_csv('location_details.csv', index=False)

# printing
print('Data has been saved to location_details.csv')

Data has been saved to location_details.csv


----

### Generating Room Allocation Dataset

----

In [67]:
def generate_room_allocation():

    # Assign room numbers and room types equally
    total_rooms = len(df_booking)

    # Generate room numbers from 101
    room_numbers = list(range(101, 101 + total_rooms))

    # Randomly assign room types: Single, Double, Suite
    room_types = np.random.choice(['Single', 'Double', 'Suite'], total_rooms)

    # Get unique countries from the location data
    countries = df_location['country'].tolist()

    # Generate room allocation data

    # Create a list to store room allocation data
    room_allocations = []
    # Iterate over the room numbers
    for i, room in enumerate(room_numbers):
        # Get booking ID from the booking data
        booking_id = df_booking.iloc[i % len(df_booking)]['booking_id']
        # Get the number of rooms booked from the booking data
        rooms_booked = df_booking.iloc[i % len(df_booking)]['rooms_booked']
        # Assign room type
        room_type = room_types[i]
        # Randomly select a country
        country = np.random.choice(countries)
        # Get hotel name based on the selected country
        hotel_name = df_location[df_location['country'] == country]['hotel_name'].values[0]
        # Append the room allocation data to the list
        room_allocations.append((room, booking_id, room_type, country, hotel_name, rooms_booked))

    # Create DataFrame from the room allocation data
    df_room_allocation = pd.DataFrame(room_allocations,
                                       columns=[
                                           'room_number',
                                            'booking_id',
                                            'room_type', 
                                            'country', 
                                            'hotel_name',
                                            'rooms_booked'
                                            ])

    return df_room_allocation

In [68]:
# Generate room allocation data
df_room_allocation = generate_room_allocation()

# Preview the dataset
df_room_allocation.head()



,room_number,booking_id,room_type,country,hotel_name,rooms_booked
0,101,23450,Suite,USA,AK Hotel,3
1,102,23451,Suite,France,AK Hotel,3
2,103,23452,Double,Australia,AK Hotel,3
3,104,23453,Double,Japan,AK Hotel,2
4,105,23454,Double,Japan,AK Hotel,1


In [69]:
df_room_allocation.groupby(['room_type', 'rooms_booked'])['room_number'].count()

room_type  rooms_booked
Double     1               117
           2               116
           3               129
Single     1               104
           2               101
           3               112
Suite      1               122
           2               108
           3               101
Name: room_number, dtype: int64

In [72]:
# Save the dataset to csv file
df_room_allocation.to_csv('room_allocation.csv', index=False)

# Print confirmation
print('Data has been saved to room_allocation.csv')

Data has been saved to room_allocation.csv


In [74]:
df_room_allocation.groupby(['room_type', 'country'])['room_number'].count()

room_type  country  
Double     Australia    79
           France       68
           Japan        56
           UK           57
           USA          74
Single     Australia    68
           France       78
           Japan        69
           UK           65
           USA          62
Suite      Australia    77
           France       68
           Japan        71
           UK           52
           USA          66
Name: room_number, dtype: int64

----

### Creating Sql connection to create database

----

In [ ]:
# Create database connection 
from sqlalchemy import create_engine

engine = create_engine(f'sqlite:///hotel1.db')

with engine.begin() as con:
    con.execute(text('DROP TABLE IF EXISTS customer_details'))
    con.execute(text('DROP TABLE IF EXISTS booking_details'))
    con.execute(text('DROP TABLE IF EXISTS location_details'))
    con.execute(text('DROP TABLE IF EXISTS room_allocation'))

# Creating tables
with engine.begin() as con:
    con.execute(text('''
        CREATE TABLE IF NOT EXISTS customer_details (
            customer_id INTEGER PRIMARY KEY,
            customer_name VARCHAR(20),
            dob DATE,
            sex VARCHAR(10),
            nationality VARCHAR(20)
        )
    '''))
    
    con.execute(text('''
        CREATE TABLE IF NOT EXISTS booking_details (
            booking_id INTEGER PRIMARY KEY, 
            room_number INTEGER,
            location_id INTEGER,
            customer_id INTEGER,
            room_type VARCHAR(20),
            room_rate INTEGER,
            rooms_booked INTEGER,
            num_persons INTEGER,
            check_in DATETIME,
            check_out DATETIME,
            num_days_stayed INTEGER,
            total_cost INTEGER,
            payment_status VARCHAR(10),
            FOREIGN KEY (customer_id) REFERENCES customer_details(customer_id),
            FOREIGN KEY (location_id) REFERENCES location_details(location_id)
        )
    '''))
    
    con.execute(text('''
        CREATE TABLE IF NOT EXISTS location_details (
            location_id INTEGER PRIMARY KEY,
            hotel_name VARCHAR(20),
            city VARCHAR(20),
            country VARCHAR(20)
                     
        )
    '''))

    con.execute(text('''
        CREATE TABLE IF NOT EXISTS room_allocation (
            room_number INTEGER PRIMARY KEY,
            booking_id INTEGER,
            room_type VARCHAR(20),
            country VARCHAR(20),
            hotel_name VARCHAR(20),
            FOREIGN KEY (booking_id) REFERENCES booking_details(booking_id)
        )
    '''))


# Remove duplicate rows based on booking_id
# df_booking = df_booking.drop_duplicates(subset=['booking_id'])

# Insert data into the database
with engine.begin() as con:
    df_customer.to_sql('customer_details', con=con, if_exists='append', index=False)
    df_booking.to_sql('booking_details', con=con, if_exists='append', index=False)
    df_location.to_sql('location_details', con=con, if_exists='append', index=False)
    df_room_allocation.to_sql('room_allocation', con=con, if_exists='append', index=False)